# co-visitation matrix

In [9]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf

In [10]:
SEED = 42
random.seed(SEED)

In [11]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [12]:
CHUNK_N = 400

In [13]:
week = "week3"

In [14]:
# データ読み込み
files = [
    "train_sessions_week1.pkl",
    "test_sessions_week1.pkl",
    "train_sessions_week2.pkl",
    "test_sessions_week2.pkl",
    "train_sessions_week3.pkl",
 #   "test_sessions_week3.pkl",
    "train_sessions_week4.pkl",
    "test_sessions_week4.pkl",
    "test_sessions.pkl"
]

dfs = []

for file in files:
    dfs.append(pd.read_pickle(PREP_DIR + file))
sessions = pd.concat(dfs)

sessions = sessions.drop(columns=["type"])
sessions["ts"] = (sessions["ts"]/1000).astype("int32")

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

del dfs
gc.collect()

0

In [15]:
dfs_outer = []
dfs_inner = []

pair_df = pd.DataFrame(columns=["aid_x", "aid_y", "cnt"])
for i, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df = df.groupby(["aid_x", "aid_y"])["session"].count().reset_index()
    df = df.rename(columns = {"session": "cnt"})
    df = df[df["cnt"]>2]
    dfs_inner.append(df.to_pandas())

    if i % 100 == 99:
        if len(dfs_inner) > 0:
            pair_df_inner = pd.concat(dfs_inner)
            pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
            dfs_outer.append(pair_df_inner)
            dfs_inner = []
            del pair_df_inner
            gc.collect()

if len(dfs_inner) > 0:
    pair_df_inner = pd.concat(dfs_inner)
    pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
    dfs_outer.append(pair_df_inner)
    del pair_df_inner
    gc.collect()

del dfs_inner
gc.collect()

pair_df = pd.concat(dfs_outer)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(20)
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_{week}.pkl")

100%|██████████| 400/400 [05:13<00:00,  1.28it/s]  


In [16]:
pair_df

,aid_x,aid_y,cnt
42,0,643097,27
33,0,532042,22
61,0,1012453,19
76,0,1350484,18
105,0,1848174,17
...,...,...,...
338333817,1855602,36026,3
338333818,1855602,54512,3
338333819,1855602,107254,3
338333820,1855602,134014,3
